# 环境和分布偏移

**前面我们学习了许多机器学习的实际应用，将模型拟合各种数据集。但是会出现一种情况，也就是训练集得分很高，测试集预测提交后发现分数很低。这种情况就很有可能是数据集之间发生了分布偏移，导致模型过拟合训练集。**

**比如说训练文件包含关于年轻的男性和女性的信息，而测试文件包含关于年老的人的信息，这意味着包含在训练和测试文件中的数据的分布是明显不同的。**

**导致数据偏移的原因，主要是样本选择的偏差，或者现实世界的变化性。**
许多失败的机器学习部署（即实际应用）都可以追究到这种方式。
有时，根据测试集的精度衡量，模型表现得非常出色。
但是当数据分布突然改变时，模型在部署中会出现灾难性的失败。
更隐蔽的是，有时模型的部署本身就是扰乱数据分布的催化剂。
*举一个有点荒谬却可能真实存在的例子。
假设我们训练了一个贷款申请人违约风险模型，用来预测谁将偿还贷款或违约。
这个模型发现申请人的鞋子与违约风险相关（穿牛津鞋申请人会偿还，穿运动鞋申请人会违约）。
此后，这个模型可能倾向于向所有穿着牛津鞋的申请人发放贷款，并拒绝所有穿着运动鞋的申请人。*

*这种情况可能会带来我们不想看到的后果。
一旦模型开始根据鞋类做出决定，顾客就会理解并改变他们的行为。
不久，所有的申请者都会穿牛津鞋，而信用度却没有相应的提高。*
总而言之，机器学习的许多应用中都存在类似的问题：
**通过将基于模型的决策引入环境，我们可能会破坏模型。**

虽然我们不可能在一节中讨论全部的问题，但我们希望揭示一些常见的问题，
并激发你的批判性思考，以便及早发现这些情况，减轻灾难性的损害。
有些解决方案很简单（要求“正确”的数据），有些在技术上很困难（实施强化学习系统），
还有一些解决方案要求我们完全跳出统计预测，解决一些棘手的、与算法伦理应用有关的哲学问题。

## 分布偏移的类型

**我们考虑数据分布可能发生变化的各种方式，以及为挽救模型性能可能采取的措施。**

首先考虑两种极端的情况。在一个经典的情景中，**假设训练数据是从某个分布$p_S(\mathbf{x},y)$中采样的，
但是测试数据将从不同分布$p_T(\mathbf{x},y)$中抽取未标记样本。**
一个清醒的现实是：**如果没有任何关于$p_S$和$p_T$之间相互关系的假设，
学习到一个分类器是不可能的。**

考虑一个二元分类问题：区分狗和猫。
如果分布可以以任意方式偏移，那么我们的情景允许病态的情况，
即**输入的分布保持不变：$p_S(\mathbf{x}) = p_T(\mathbf{x})$，
但标签全部翻转：$p_S(y | \mathbf{x}) = 1 - p_T(y | \mathbf{x})$。**
换言之，如果将来所有的“猫”现在都是狗，而我们以前所说的“狗”现在是猫。
而此时输入$p(\mathbf{x})$的分布没有任何改变，
**那么我们就可以将这种情况转化为完全没有变化的情景。**

幸运的是，在对未来我们的数据可能发生变化的一些限制性假设下，
有些算法可以检测这种偏移，甚至可以动态调整，提高原始分类器的精度。

### 协变量偏移

**根据贝叶斯公式： $p(\mathbf{x},\mathbf{y})= p(x |\mathbf{y})p(\mathbf{y})= p(y |\mathbf{x})p(\mathbf{x})$，
其中，输入空间的概率分布$p(\mathbf{x})$，输出空间的分布$p(\mathbf{y})$，以及表示该机器学习任务的条件概率分布$p(y |\mathbf{x})$，
当这三者之一发生了变化，我们都认为数据集的分布发生了偏移。**

在不同分布偏移中，**协变量偏移可能是最常见**的。
这里我们假设：**虽然输入的分布$p(\mathbf{x})$可能随时间而改变，
但标签函数（即条件分布$P(y \mid \mathbf{x})$）没有改变。
也就是说，虽然训练集和测试集的输入服从不同分布，但背后是服从同一个函数关系。**
统计学家称之为*协变量偏移*（covariate shift），
因为这个问题是由于协变量（特征）分布的变化而产生的。
虽然有时我们可以在不引用因果关系的情况下对分布偏移进行推断，
但**在我们认为$\mathbf{x}$导致$y$的情况下，协变量偏移是一种十分符合的假设。**

考虑一下区分猫和狗的问题：训练数据包括 :numref:`fig_cat-dog-train`中的图像。

![区分猫和狗的训练数据](./cat-dog-train.svg)
:label:`fig_cat-dog-train`

在测试时，我们被要求对 :numref:`fig_cat-dog-test`中的图像进行分类。

![区分猫和狗的测试数据](./cat-dog-test.svg)
:label:`fig_cat-dog-test`

训练集由真实照片组成，而测试集只包含卡通图片。
假设在一个与测试集的特征有着本质不同的数据集上进行训练，
如果没有方法来适应新的领域，可能会有麻烦。

### 标签偏移

*标签偏移*（label shift）描述了与协变量偏移相反的问题。
这里我们**假设标签边缘概率$P(y)$可以改变，
但是类别条件分布$P(\mathbf{x} \mid y)$在不同的领域之间保持不变，也就是目标变量的偏移。
当我们认为$y$导致$\mathbf{x}$时，标签偏移是一个合理的假设。**
例如，预测患者的疾病，我们可能根据症状来判断，
即使疾病的相对流行率随着时间的推移而变化。
标签偏移在这里是恰当的假设，因为疾病会引起症状。

在另一些情况下，标签偏移和协变量偏移假设可以同时成立。
例如，当标签是确定的，即使$y$导致$\mathbf{x}$，协变量偏移假设也会得到满足。
有趣的是，在这些情况下，使用基于标签偏移假设的方法通常是有利的。
这是因为这些方法倾向于包含看起来像标签（通常是低维）的对象，
而不是像输入（通常是高维的）对象。

### 概念偏移

我们也可能会遇到*概念偏移*（concept shift）：
**当标签的定义发生变化时，就会出现这种问题。**
这听起来很奇怪——一只猫就是一只猫，不是吗？
然而，其他类别会随着不同时间的用法而发生变化。
精神疾病的诊断标准、所谓的时髦、以及工作头衔等等，都是概念偏移的日常映射。
事实证明，假如我们环游美国，根据所在的地理位置改变我们的数据来源，
我们会发现关于“软饮”名称的分布发生了相当大的概念偏移，
如 :numref:`fig_popvssoda` 所示。


<img src="popvssoda.png" alt="Drawing" style="width:400px" >

如果我们要建立一个机器翻译系统，
$P(y \mid \mathbf{x})$**的分布可能会因我们的位置不同而得到不同的翻译。
这个问题可能很难被发现。
所以，我们最好可以利用在时间或空间上逐渐发生偏移的知识，来随时修改模型。**

## 分布偏移纠正

正如我们所讨论的，在许多情况下训练和测试分布$P(\mathbf{x}, y)$是不同的。
在一些情况下，我们很幸运，不管协变量、标签或概念如何发生偏移，模型都能正常工作。
在另一些情况下，我们**想要通过运用策略来应对这种偏移**，从而做得更好。
本节的其余部分将着重于应对这种偏移的技术细节。

### 经验风险与实际风险
:label:`subsec_empirical-risk-and-risk`

首先我们反思一下在模型训练期间到底发生了什么？
训练数据$\{(\mathbf{x}_1, y_1), \ldots, (\mathbf{x}_n, y_n)\}$
的特征和相关的标签经过迭代，在每一个小批量之后更新模型$f$的参数。
为了简单起见，我们不考虑正则化，因此极大地降低了训练损失：

$$\mathop{\mathrm{minimize}}_f \frac{1}{n} \sum_{i=1}^n l(f(\mathbf{x}_i), y_i),$$


其中$l$是损失函数，用来度量：
给定标签$y_i$，预测$f(\mathbf{x}_i)$的“糟糕程度”。
统计学家称上式这一项为经验风险。
***经验风险***（empirical risk）**是为了近似 *真实风险***（true risk），
整个训练数据上的平均损失，即**从其真实分布$p(\mathbf{x},y)$中
抽取的所有数据的总体损失的期望值**：

$$E_{p(\mathbf{x}, y)} [l(f(\mathbf{x}), y)] = \int\int l(f(\mathbf{x}), y) p(\mathbf{x}, y) \;d\mathbf{x}dy.$$
:eqlabel:`eq_true-risk`

**然而在实践中，我们通常无法获得总体数据。**
因此，*经验风险最小化*即在 :eqref:`eq_empirical-risk-min`中**最小化经验风险，
是一种实用的机器学习策略，希望能近似最小化真实风险。**

### 协变量偏移纠正
:label:`subsec_covariate-shift-correction`

**目标是使用密度比的重要性加权，去除产生协变量偏移现象且不重要的特征。**

**因此，我们需要解决两个问题：
第一个问题是关于如何区分来自两个分布的数据；
第二个问题是关于加权经验风险的最小化问题。
在这个问题中，我们将对其中的项加权$\beta_i$，并研究$\beta_i$的求法。**

#### 解决第一个问题（区分数据来源）

假设对于带标签的数据$(\mathbf{x}_i, y_i)$，
我们**要评估$P(y \mid \mathbf{x})$。
观测值$\mathbf{x}_i$是从某些*源分布*$q(\mathbf{x})$中得出的，**
而不是从*目标分布*$p(\mathbf{x})$中得出的。
幸运的是，依赖性假设意味着**条件分布保持不变，即：
$p(y \mid \mathbf{x}) = q(y \mid \mathbf{x})$。**
如果源分布$q(\mathbf{x})$是“错误的”，
我们可以通过在真实风险的计算中，**使用总体损失期望值来进行纠正**：

$$
\begin{aligned}
\int\int l(f(\mathbf{x}), y) p(y \mid \mathbf{x})p(\mathbf{x}) \;d\mathbf{x}dy =
\int\int l(f(\mathbf{x}), y) q(y \mid \mathbf{x})q(\mathbf{x})\frac{p(\mathbf{x})}{q(\mathbf{x})} \;d\mathbf{x}dy.
\end{aligned}
$$

换句话说，**我们需要根据数据来自正确分布与来自错误分布的概率之比，
来重新衡量每个数据样本的权重**：

$$\beta_i \stackrel{\mathrm{def}}{=} \frac{p(\mathbf{x}_i)}{q(\mathbf{x}_i)}.$$

将权重$\beta_i$代入到每个数据样本$(\mathbf{x}_i, y_i)$中，
我们可以**使用”加权经验风险最小化“来训练模型**：

$$\mathop{\mathrm{minimize}}_f \frac{1}{n} \sum_{i=1}^n \beta_i l(f(\mathbf{x}_i), y_i).$$
:eqlabel:`eq_weighted-empirical-risk-min`

**由于不知道这个比率，我们需要估计它**。
有许多方法都可以用，包括一些花哨的算子理论方法，
试图直接使用最小范数或最大熵原理重新校准期望算子。
对于任意一种这样的方法，我们都需要从两个分布中抽取样本：
“真实”的分布$p$，通过访问测试数据获取；
训练集$q$，通过人工合成的很容易获得。
请注意，我们只需要特征$\mathbf{x} \sim p(\mathbf{x})$，
不需要访问标签$y \sim p(y)$。

在这种情况下，**有一种非常有效的方法可以得到几乎与原始方法一样好的结果：
*对数几率回归*（logistic regression），也就是利用sigmoid函数对数据进行分类，判断数据是正确的还是错误的。**

$$\operatorname{sigmoid}(x) = \frac{1}{1 + \exp(-x)}.$$

这是用于二元分类的softmax回归（见 :numref:`sec_softmax`）的一个特例。
综上所述，我们**学习了一个分类器来区分从$p(\mathbf{x})$抽取的数据
和从$q(\mathbf{x})$抽取的数据**。
如果无法区分这两个分布，则意味着相关的样本可能来自这两个分布中的任何一个。
另一方面，任何可以很好区分的样本都应该相应地显著增加或减少权重。

#### 解决第二个问题（加权经验风险最小化）

为了简单起见，假设我们**分别从$p(\mathbf{x})$和$q(\mathbf{x})$
两个分布中抽取相同数量的样本。**
现在用$z$标签表示：从$p$抽取的数据为$1$，从$q$抽取的数据为$-1$。
然后，混合数据集中的概率由下式给出

$$P(z=1 \mid \mathbf{x}) = \frac{p(\mathbf{x})}{p(\mathbf{x})+q(\mathbf{x})} \text{ and hence } \frac{P(z=1 \mid \mathbf{x})}{P(z=-1 \mid \mathbf{x})} = \frac{p(\mathbf{x})}{q(\mathbf{x})}.$$

因此，如果我们使用对数几率回归方法，其中
$P(z=1 \mid \mathbf{x})=\frac{1}{1+\exp(-h(\mathbf{x}))}$
（$h$是一个参数化函数），则很自然有：

$$
\beta_i = \frac{1/(1 + \exp(-h(\mathbf{x}_i)))}{\exp(-h(\mathbf{x}_i))/(1 + \exp(-h(\mathbf{x}_i)))} = \exp(h(\mathbf{x}_i)).
$$



现在，我们来看一下完整的协变量偏移纠正算法。
假设我们有一个训练集$\{(\mathbf{x}_1, y_1), \ldots, (\mathbf{x}_n, y_n)\}$
和一个未标记的测试集$\{\mathbf{u}_1, \ldots, \mathbf{u}_m\}$。
对于协变量偏移，我们假设$1 \leq i \leq n$的$\mathbf{x}_i$来自某个源分布，
$\mathbf{u}_i$来自目标分布。
以下是**纠正协变量偏移的典型算法：**

1. 生成一个二元分类训练集：$\{(\mathbf{x}_1, -1), \ldots, (\mathbf{x}_n, -1), (\mathbf{u}_1, 1), \ldots, (\mathbf{u}_m, 1)\}$。
1. 用对数几率回归训练二元分类器得到函数$h$。
1. 使用$\beta_i = \exp(h(\mathbf{x}_i))$或更好的$\beta_i = \min(\exp(h(\mathbf{x}_i)), c)$（$c$为常量）对训练数据进行加权。
1. 使用权重$\beta_i$进行加权风险最小化中$\{(\mathbf{x}_1, y_1), \ldots, (\mathbf{x}_n, y_n)\}$的训练。

**请注意，上述算法依赖于一个重要的假设：
需要目标分布(例如，测试分布)中的每个数据样本在训练时出现的概率非零。**
如果我们找到$p(\mathbf{x}) > 0$但$q(\mathbf{x}) = 0$的点，
那么相应的重要性权重会是无穷大。



### 标签偏移纠正

假设我们处理的**是$k$个类别的分类任务**。
使用与协变量偏移纠正中中相同符号，
$q$和$p$中分别是源分布（例如训练时的分布）和目标分布（例如测试时的分布）。
假设**标签的分布随时间变化：$q(y) \neq p(y)$，
但类别条件分布保持不变：$q(\mathbf{x} \mid y)=p(\mathbf{x} \mid y)$。**
如果源分布  是“错误的”，
我们可以根据（4.9.2）中定义的真实风险中的恒等式进行更正：

$$
\begin{aligned}
\int\int l(f(\mathbf{x}), y) p(\mathbf{x} \mid y)p(y) \;d\mathbf{x}dy =
\int\int l(f(\mathbf{x}), y) q(\mathbf{x} \mid y)q(y)\frac{p(y)}{q(y)} \;d\mathbf{x}dy.
\end{aligned}
$$

这里，重要性权重将对应于标签似然比率

$$\beta_i \stackrel{\mathrm{def}}{=} \frac{p(y_i)}{q(y_i)}.$$

下面的目标仍然是求$\beta_i$

**标签偏移的一个好处是，如果我们在源分布上有一个相当好的模型，
那么我们可以得到对这些权重的一致估计，而不需要处理周边的其他维度。**
在深度学习中，输入往往是高维对象（如图像），而标签通常是低维（如类别）。

为了估计目标标签分布$p(y)$，我们首先采用性能相当好的现成的分类器（通常基于训练数据进行训练），
并**使用验证集（也来自训练分布）计算其混淆矩阵。
混淆矩阵$\mathbf{C}$是一个$k \times k$矩阵，
其中每列对应于标签类别，每行对应于模型的预测类别。
每个单元格的值$c_{ij}$是验证其中真实标签为$j$，
而我们的模型预测为$i$的样本数量所占的比例。**

但是，我们无法直接计算目标数据上的混淆矩阵，
因为我们无法看到真实环境下的样本的标签，
除非我们再搭建一个复杂的实时标注流程。

然而，我们所能做的是**将所有模型在测试时的预测取平均数，
得到平均模型输出$\mu(\hat{\mathbf{y}}) \in \mathbb{R}^k$，
其中第$i$个元素$\mu(\hat{y}_i)$是我们模型预测测试集中$i$的总预测分数。**

结果表明，**如果（1）我们的分类器一开始就相当准确，
并且（2）目标数据只包含我们以前见过的类别，
以及（3）如果标签偏移假设成立（这里最强的假设），
我们就可以通过求解一个简单的线性系统来估计测试集的标签分布**

$$\mathbf{C} p(\mathbf{y}) = \mu(\hat{\mathbf{y}}),$$

因为作为一个估计，$\sum_{j=1}^k c_{ij} p(y_j) = \mu(\hat{y}_i)$
对所有$1 \leq i \leq k$成立，
其中$p(y_j)$是$k$维标签分布向量$p(\mathbf{y})$的第$j^\mathrm{th}$元素。
**如果我们的分类器一开始就足够精确，那么混淆矩阵$\mathbf{C}$将是可逆的，
进而我们可以得到一个解$p(\mathbf{y}) = \mathbf{C}^{-1} \mu(\hat{\mathbf{y}})$。**

因为我们观测源数据上的标签，所以**很容易估计分布$q(y)$。**
那么对于标签为$y_i$的任何训练样本$i$，
我们可以使用我们估计的$p(y_i)/q(y_i)$比率来**计算权重$\beta_i$**，
并将其代入 :eqref:`eq_weighted-empirical-risk-min`中的加权经验风险最小化中。

### 概念偏移纠正

概念偏移很难用原则性的方式解决。
例如，在一个问题突然从“区分猫和狗”偏移为“区分白色和黑色动物”的情况下，
除了从零开始收集新标签和训练，别无妙方。
幸运的是，在实践中这种极端的偏移是罕见的。
相反，通常情况下，概念的变化总是缓慢的。
比如下面是一些例子：

* 在计算广告中，新产品推出后，旧产品变得不那么受欢迎了。这意味着广告的分布和受欢迎程度是逐渐变化的，任何点击率预测器都需要随之逐渐变化。
* 由于环境的磨损，交通摄像头的镜头会逐渐退化，影响摄像头的图像质量。
* 新闻内容逐渐变化（即新新闻的出现）。

在这种情况下，我们可以使用与训练网络相同的方法，使其适应数据的变化。
换言之，我们**使用新数据更新现有的网络权重**，而不是从头开始训练。




## 小结

* 在许多情况下，训练集和测试集并不来自同一个分布。这就是所谓的分布偏移。
* 真实风险是从真实分布中抽取的所有数据的总体损失的预期。然而，这个数据总体通常是无法获得的。经验风险是训练数据的平均损失，用于近似真实风险。在实践中，我们进行经验风险最小化。
* 在相应的假设条件下，可以在测试时检测并纠正协变量偏移和标签偏移。在测试时，不考虑这种偏移可能会成为问题。
* 在某些情况下，环境可能会记住自动操作并以令人惊讶的方式做出响应。在构建模型时，我们必须考虑到这种可能性，并继续监控实时系统，并对我们的模型和环境以意想不到的方式纠缠在一起的可能性持开放态度。

## 练习

1. 当我们改变搜索引擎的行为时会发生什么？用户可能会做什么？广告商呢？
2. 实现一个协变量偏移检测器。提示：构建一个分类器。
3. 实现协变量偏移纠正。
4. 除了分布偏移，还有什么会影响经验风险接近真实风险的程度？

[Discussions](https://discuss.d2l.ai/t/1822)
